In [2]:
!pip install duckdb uuid pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 124.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 139.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 128.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.5/503.5 kB 108.3 MB/s eta 0:00:00
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6480 sha256=0afa824b06031d8df9bca547c53a1f47742e23ca251359ae69bd0ffa0b691a37
  Stored in directory: /home/gitpod/.cache/pip/wheels/1b/6c/cb/f9aae2bc97333c3d6e060826c1ee9e44e46306a178e5783505
Successfully built uuid


In [3]:
import pandas as pd
import mmap
import os
import duckdb
import uuid
import json
import glob

In [27]:
!cat ../data/xxx/xaa | ../build/json-buffet > extracted_npi_offsets.csv

In [15]:
fbuff = open("../data/xxx/xaa", mode="r", encoding="utf8")

In [29]:
df = pd.read_csv('./extracted_npi_offsets.csv', names=['NPI', 'offset', 'size'])

In [30]:
df

,NPI,offset,size
0,1962697342,99985389,4510
1,1992023816,99985389,4510
2,1861835290,99985389,4510
3,1801865787,99985389,4510
4,1861699548,99985389,4510
...,...,...,...
5424824,1154400513,84086296,599607
5424825,1174502934,59477315,1422148
5424826,1710414560,45976282,6567
5424827,1720224801,59477315,1422148


In [40]:
NPI = 1720224801
NPI_DB = f'{NPI}_indexed.db'

In [41]:
con = duckdb.connect(database=NPI_DB, read_only=False)
con.execute("CREATE OR REPLACE TABLE extract(id UUID default uuid(), file VARCHAR, extracted_at TIMESTAMP, offs INTEGER, size INTEGER)")
con.execute("CREATE OR REPLACE TABLE provider_groups(tin VARCHAR, type VARCHAR)")
con.execute("CREATE OR REPLACE TABLE npi_tin(npi VARCHAR, tin VARCHAR)")
con.execute("CREATE OR REPLACE TABLE npi_extract(npi VARCHAR, extract UUID)")
con.execute("CREATE OR REPLACE TABLE negotiated_rates(id UUID default uuid(), tin VARCHAR, negotiated_type VARCHAR, negotiation_rate DOUBLE, expiration_date TIMESTAMP, service_code VARCHAR[], billing_class VARCHAR, billing_code_modifier VARCHAR[], additional_information VARCHAR)")
con.execute("CREATE OR REPLACE TABLE npi_negotiated_rates(negotiate_rate_id UUID, npi VARCHAR)")


In [43]:
for row in df[df["NPI"] == NPI].to_records():
    print("Reading record")
    fbuff.seek(row[2], 0)
    data = fbuff.read(row[3])
    obj = json.loads(data)
    pgs = []
    pg_tin = {}
    npi_tins = []
    negotiated_rates = []
    npi_negotiated_rates = []

    provider_groups = obj['provider_groups']
    #print(obj
    for pg in provider_groups:
        tin = pg['tin']['value']
        npis = pg['npi']

       # print(pg)
        if (pg_tin.get(tin) != True):
            pgs.append([tin, pg['tin']['type']])
            pg_tin[tin] = True

        for npi in npis:
            npi_tins.append([npi, tin])
    for nr in obj['negotiated_prices']:
        nr_id = str(uuid.uuid4())
        negotiated_rates.append([
            nr_id,
            tin,
            nr['negotiated_type'],
            nr['negotiated_rate'],
            nr['expiration_date'],
            nr['service_code'],
            nr['billing_class'],
            nr['billing_code_modifier'],
            nr['additional_information']
        ])
        for npi in npi_tins:
            npi_negotiated_rates.append([
                nr_id,
                npi[0]
            ])
    con.executemany('INSERT INTO provider_groups VALUES (?, ?)', pgs)
    con.executemany('INSERT INTO npi_tin VALUES (?, ?)', npi_tins)
    con.executemany('INSERT INTO negotiated_rates VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', negotiated_rates)

Reading record


In [44]:
!ls -lahrt $NPI_DB

-rw-r--r-- 1 gitpod gitpod 12K Jul 12 16:42 1720224801_indexed.db


In [34]:
con.execute("select count(distinct npi) from npi_tin").fetchall()

[(340,)]

In [35]:
con.execute("select avg(negotiated_rate) from negotiated_rates").fetchall()

[(UUID('864f1a2c-675a-4c5c-ba3d-3fa41d076bbb'),
  '204435701',
  'fee schedule',
  356.99,
  datetime.datetime(9999, 12, 31, 0, 0),
  ['02',
   '05',
   '06',
   '07',
   '08',
   '19',
   '21',
   '22',
   '23',
   '24',
   '26',
   '31',
   '34',
   '41',
   '42',
   '51',
   '52',
   '53',
   '56',
   '61'],
  'professional',
  [''],
  ''),
 (UUID('654c116b-ac7e-482f-bfdf-f19f51b19d41'),
  '204435701',
  'fee schedule',
  478.36,
  datetime.datetime(9999, 12, 31, 0, 0),
  ['01',
   '03',
   '04',
   '09',
   '11',
   '12',
   '13',
   '14',
   '15',
   '16',
   '17',
   '18',
   '20',
   '25',
   '32',
   '33',
   '49',
   '50',
   '54',
   '55',
   '57',
   '60',
   '62',
   '65',
   '71',
   '72',
   '81',
   '99'],
  'professional',
  [''],
  '')]

In [111]:
con.execute("DELETE FROM provider_groups")
con.execute("DELETE FROM npi_tin")
con.execute("DELETE FROM negotiated_rates")
con.execute("DELETE FROM npi_negotiated_rates")